# LLM Code Comparison Benchmark (eval the eval)

Measures how often the LLM correctly predicts whether a PR will resolve an issue, should be useful for prompt optimization and seeing how accurate LLM code comparison is relative to running tests.

Compares SWEBench-submitted patches to ground truth patches, and sees if the LLM can guess whether the submitted patch would resolve the issue.

### Envars:
- `OPENAI_API_KEY` for LLM calls

In [3]:
!pip install -q datasets openai datasets
!git clone https://github.com/swe-bench/experiments.git

Cloning into 'experiments'...
remote: Enumerating objects: 2911, done.
remote: Counting objects: 100% (1667/1667), done.
remote: Compressing objects: 100% (1094/1094), done.
remote: Total 2911 (delta 631), reused 740 (delta 572), pack-reused 1244 (from 1)
Receiving objects: 100% (2911/2911), 64.70 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (1046/1046), done.
Updating files: 100% (2172/2172), done.


In [5]:
from pathlib import Path
import json
import random
from datasets import load_dataset


eval_dir = Path("experiments/evaluation/lite")
ds = load_dataset("princeton-nlp/SWE-bench_Lite")

/Users/alex/workspace/trading-projects/bittensor-projects/taoception/taoception-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from dataclasses import dataclass
from typing import *
import datasets


@dataclass
class IssueSubmission:
    instance_id: str
    problem_statement: str
    model_patch: str
    gold_patch: str
    resolved_state: str
    submission_dir: str

datasets.logging.set_verbosity(datasets.logging.CRITICAL)

instance_to_state = {}

def get_instances(max_items: Optional[int] = None, random_shuffle: bool = True) -> List[IssueSubmission]:
    instances = []

    all_dirs = list(eval_dir.iterdir())
    if random_shuffle:
        random.shuffle(all_dirs)

    for current_dir in all_dirs:
        print(f"Using SWEBench submission at {current_dir}")

        print("Loading results.json...")
        with open(current_dir / "results/results.json", "r") as file:
            swebench_results = json.load(file)
        print("Finished loading results.json")
    
        # Amend reverse map
        for state, instance_ids in swebench_results.items():
            for iid in instance_ids:
                instance_to_state[iid] = state

        print("Loading submitted preds...")
        with open(current_dir / "all_preds.jsonl", "r") as file:
            submitted_preds = [json.loads(line) for line in file]
        print(f"Loadeded submitted preds, length={len(submitted_preds)}")

        for submitted_pred in submitted_preds:
            instance_id, model_patch = submitted_pred["instance_id"], submitted_pred["model_patch"]
            resolved_state = instance_to_state[instance_id]

            issue_row = ds["test"].filter(lambda row: row["instance_id"] == instance_id)[0]
            gold_patch, problem_statement = issue_row["patch"], issue_row["problem_statement"]

            # print("Adding instance of id ", instance_id)
            instances.append(IssueSubmission(instance_id, problem_statement, model_patch, gold_patch, resolved_state, current_dir))

            if max_items is not None and len(instances) > max_items:
                print(f"Reached limit of {max_items} instances. Returning...")
                return instances
        print(f"Reached end of current dir {current_dir}. Instances: {len(instances)}")

    return instances

In [60]:
import os
from collections import Counter
from openai import OpenAI

client = OpenAI()

def eval_issue_submission(llm_prompt_template: str, issue_submission: IssueSubmission) -> int:
    prompt = llm_prompt_template.format(
        model_patch=issue_submission.model_patch,
        gold_patch=issue_submission.gold_patch,
        problem_statement=issue_submission.problem_statement
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
    )
    comparison = response.choices[0].message.content
    return comparison[-1]


def eval_prompt(llm_prompt_template: str, num_issues: int = 10, should_log=True) -> None:
    # Gather data
    instances = get_instances(max_items=num_issues, random_shuffle=False)
    
    # Eval data
    outputs = {}
    for instance in instances:
        try:
            eval_output = eval_issue_submission(llm_prompt_template, instance)
            if should_log:
                print(f"Issue {instance.instance_id} with state {instance.resolved_state}, model output={eval_output}")
            
            if eval_output.isdigit() and int(eval_output) in [0, 1]:
                outputs[instance.instance_id] = int(eval_output)
            else:
                print(f"Skipping eval of issue {instance.instance_id} because model output ended with '{eval_output}' instead of 0 or 1")
        
        except Exception as e:
            print(f"Ran into error: {e}")
            continue
    return outputs


In [63]:
LLM_TEMPLATE_1 = """
You are being asked to identify important differences between 2 solutions to a given problem. The problem is as follows:
```
{problem_statement}
```
The 1st solution is:
```
{model_patch}
```
The second solution is:
```
{gold_patch}
```
First, list all the differences between these patches of code that you can think of. 
Then, let's assume that the second patch is a correct solution to the problem. 
Based on the similarity to this patch and the differences you outlined, would you expect the first patch to also be a correct solution? 
Reason about why or why not. 
Then, output a 1 if you think the first patch will address the problem, and a 0 if you think it will not. 
Do not use markdown, and make sure the digit is the last thing you output.
"""

print("Prompt #1:")
outputs = eval_prompt(LLM_TEMPLATE_1, num_issues=100, should_log=False)

# Print summary
zeros = Counter(instance_to_state[instance_id] for instance_id, score in outputs.items() if score == 0)
ones = Counter(instance_to_state[instance_id] for instance_id, score in outputs.items() if score == 1)

print("\nEval results:")
print("    zeros", zeros)
print("    ones", ones)

Prompt #1:
Using SWEBench submission at experiments/evaluation/lite/20240612_MASAI_gpt4o
Loading results.json...
Finished loading results.json
Loading submitted preds...
Loadeded submitted preds, length=287
Reached limit of 100 instances. Returning...
Skipping eval of issue sympy__sympy-20154 because model output ended with '.' instead of 0 or 1
Skipping eval of issue django__django-16408 because model output ended with '.' instead of 0 or 1
Skipping eval of issue astropy__astropy-12907 because model output ended with '.' instead of 0 or 1

Eval results:
    zeros Counter({'applied': 49, 'resolved': 5})
    ones Counter({'resolved': 25, 'applied': 19})
